In [1]:
from preprocessing import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
X_train, X_test, Y_test, Y_train = svm()
display(X_train)


In [ ]:
X_train, X_test, Y_test, Y_train = svm()

In [ ]:
model = LinearSVC()
model.fit(X_train, Y_train, random_state = 123)

metrics = []
for c in range(1, 1000, 2):
    clf = LinearSVC(C=c)
    clf.fit(X_train, y_train)
    metrics.append((c, (clf.predict(X_train) == Y_train).sum()))

pd.DataFrame(metrics, columns=['c', 'correct_in_val-dev']).plot(x='c')